# Implementing an SCD Type 2 dimension from a CDC source using Snowflakes's Stored procedure and Data Quality Checks 

<img src = "img/stored_procedure_scd_type_2.jpg">

## SCD Type 2 Table

### Source Data (PRODUCT_STATUS_CDC table)

```sql
-- Step 0: Check the CDC table
SELECT * FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_CDC;
```

| PRODUCT_KEY | STATUS | CHANGE_TYPE | CHANGE_TIME           | CDC_LOG_POSITION |
|-------------|--------|-------------|------------------------|-------------------|
| 1           | 10     | INSERT      | 2019-01-01 10:00:00   | 1                 |
| 1           | 10     | INSERT      | 2019-01-01 10:00:00   | 1                 |
| 1           | 10     | UPDATE      | 2019-01-01 10:30:00   | 2                 |
| 1           | 20     | UPDATE      | 2019-01-01 11:00:00   | 3                 |
| 1           | 20     | DELETE      | 2019-01-01 12:00:00   | 4                 |
| 1           | 10     | INSERT      | 2019-01-01 14:00:00   | 5                 |

### CTE 1: deduplicated_cdc
* Deduplicates rows where all fields (excluding CDC_LOG_POSITION) are identical. Here, the first two rows are duplicates, so one is removed.

```sql
-- Step 1: Deduplicate records from the CDC stream
-- Here, we deduplicate the CDC table, selecting only unique records from the CDC table
-- Solved issue: "at least once semantics" ("We may receive the same exact cdc record twice or more times")
deduplicated_cdc AS (
        SELECT DISTINCT
            product_key,
            status,
            json_extract_path_text(change_type, 'type') AS change_type,
            change_time,
            cdc_log_position
        FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_CDC
        WHERE change_time IS NOT NULL
)
```

| PRODUCT_KEY | STATUS | CHANGE_TYPE | CHANGE_TIME           | CDC_LOG_POSITION |
|-------------|--------|-------------|------------------------|-------------------|
| 1           | 10     | INSERT      | 2019-01-01 10:00:00   | 1                 |
| 1           | 10     | UPDATE      | 2019-01-01 10:30:00   | 2                 |
| 1           | 20     | UPDATE      | 2019-01-01 11:00:00   | 3                 |
| 1           | 20     | DELETE      | 2019-01-01 12:00:00   | 4                 |
| 1           | 10     | INSERT      | 2019-01-01 14:00:00   | 5                 |

### CTE 2: hash_table_1
* Computes _hash_1 for each row based on the STATUS and CHANGE_TYPE.
* Sets _valued_changed_1 to TRUE for rows where _hash_1 differs from the previous row’s _hash_1.

```sql
-- Step 2: Detect the changes that need to be captured for a given product_key
-- Here, we want to track changes in status for a given product
-- For a given product:
-- If inserted or updated, it will be included in the SCD.
-- If it was deleted, it also must be tracked in the SCD, but in this case hash needs to include the 'cdc_log_position' to differentiate (CASE method below) or we won't capture this case in the value_changed column   
hash_table_1 AS (
    SELECT *,
        -- HASH(product_key, status) AS _hash_test,
        CASE 
            WHEN change_type != 'DELETE' THEN HASH(product_key, status) -- Same hash for the same status if not a DELETE
            ELSE HASH(product_key, status, change_type)                 -- Different hash if DELETE
        END AS _hash_1,
        LAG(_hash_1) OVER (PARTITION BY product_key ORDER BY cdc_log_position) AS _previous_hash_1, -- Compare hash within the same product_key
        -- COALESCE(_hash_test != _previous_hash_1, TRUE) AS _valued_changed_test,
        COALESCE(_hash_1 != _previous_hash_1, TRUE) AS _valued_changed_1 -- Flag if there is an effective change
    FROM deduplicated_cdc
)
```

| PRODUCT_KEY | STATUS | CHANGE_TYPE | CHANGE_TIME           | CDC_LOG_POSITION | _hash_1      | _previous_hash_1 | _valued_changed_1 |
|-------------|--------|-------------|------------------------|-------------------|--------------|------------------|--------------------|
| 1           | 10     | INSERT      | 2019-01-01 10:00:00   | 1                 | hash(10)     | NULL             | TRUE               |
| 1           | 10     | UPDATE      | 2019-01-01 10:30:00   | 2                 | hash(10)     | hash(10)         | FALSE              |
| 1           | 20     | UPDATE      | 2019-01-01 11:00:00   | 3                 | hash(20)     | hash(10)         | TRUE               |
| 1           | 20     | DELETE      | 2019-01-01 12:00:00   | 4                 | hash(20, del)| hash(20)         | TRUE               |
| 1           | 10     | INSERT      | 2019-01-01 14:00:00   | 5                 | hash(10)     | hash(20, del)    | TRUE               |


### CTE 3: hash_table_2
* Filters rows to retain only those where _valued_changed_1 is TRUE

```sql
-- Step 3: Intermediate table before the SCD logic 
-- Here, for a given product, we filter for all historical rows where changes might have happened (the ones that need to be present in the SCD table)
-- We made sure to also include the DELETE cases
-- We also brought the change_time as an initial step towards scd_start_time and scd_end_time
hash_table_2 AS (
    SELECT * exclude (_hash_1, _previous_hash_1, _valued_changed_1)
    FROM hash_table_1 
    WHERE _valued_changed_1 = 'TRUE'
)
```

| PRODUCT_KEY | STATUS | CHANGE_TYPE | CHANGE_TIME           | CDC_LOG_POSITION |
|-------------|--------|-------------|------------------------|-------------------|
| 1           | 10     | INSERT      | 2019-01-01 10:00:00   | 1                 |
| 1           | 20     | UPDATE      | 2019-01-01 11:00:00   | 3                 |
| 1           | 20     | DELETE      | 2019-01-01 12:00:00   | 4                 |
| 1           | 10     | INSERT      | 2019-01-01 14:00:00   | 5                 |

### CTE 4: final_table
* Adds the is_active column, where DELETE rows are marked is_active = FALSE.

```sql
-- Step 4: Create is_active column based on the previous status and current change_type
-- Here, we want to flag DELETEs as not active because all we need is to close the previous status, so don't need to keep the row where change_type = DELETE
-- For now, we will not filter the IS_ACTIVE = TRUE values because we need the DELETE rows to close the previous record
final_table AS (
    SELECT *,
        -- Use LAG to get the previous status in this step
        LAG(status) OVER (PARTITION BY product_key ORDER BY change_time) AS _previous_status, -- Get previous status
        -- Set is_active to FALSE if the previous status is the same and change_type is DELETE
        CASE
            WHEN change_type = 'DELETE' THEN FALSE
            ELSE TRUE
        END AS is_active
    FROM hash_table_2
)
```

| PRODUCT_KEY | STATUS | CHANGE_TYPE | CHANGE_TIME           | CDC_LOG_POSITION | is_active |
|-------------|--------|-------------|------------------------|-------------------|-----------|
| 1           | 10     | INSERT      | 2019-01-01 10:00:00   | 1                 | TRUE      |
| 1           | 20     | UPDATE      | 2019-01-01 11:00:00   | 3                 | TRUE      |
| 1           | 20     | DELETE      | 2019-01-01 12:00:00   | 4                 | FALSE     |
| 1           | 10     | INSERT      | 2019-01-01 14:00:00   | 5                 | TRUE      |

### CTE 5: scd_table
* Calculates scd_start_time and scd_end_time. The scd_end_time uses LEAD to set the end time to the next CHANGE_TIME (or default future date if none).
* The DELETE row closes the previous status

```sql
-- Step 5: Opening and Closing rows (scd_start_time and scd_end_time) 
-- Here, we close previous records with the respective scd_start_time and scd_end_time based on change_time
scd_table AS (
    SELECT 
        product_key                        AS product_status_product_key, -- Foreign key to product_dm table
        status                             AS product_status_status,      -- Status of the product for this SCD record
        CAST(change_time AS TIMESTAMP_NTZ) AS scd_start_time,             -- Start time of the SCD record in TIMESTAMP_NTZ
        -- Set end time based on next change_time or default to '2999-01-01 00:00:00' if no further changes
        CASE 
            WHEN is_active = FALSE THEN CAST(change_time AS TIMESTAMP_NTZ)
            ELSE CAST(
                COALESCE(
                    LEAD(change_time) OVER (PARTITION BY product_key ORDER BY change_time),
                    TIMESTAMP '2999-01-01 00:00:00'
                ) AS TIMESTAMP_NTZ
            )
        END AS scd_end_time,
        is_active,
        cdc_log_position
    FROM final_table
)
```

| product_status_product_key | product_status_status | scd_start_time         | scd_end_time           | is_active |
|----------------------------|-----------------------|-------------------------|-------------------------|-----------|
| 1                          | 10                    | 2019-01-01 10:00:00    | 2019-01-01 11:00:00     | TRUE      |
| 1                          | 20                    | 2019-01-01 11:00:00    | 2019-01-01 12:00:00     | TRUE      |
| 1                          | 20                    | 2019-01-01 12:00:00    | 2019-01-01 12:00:00     | FALSE     |
| 1                          | 10                    | 2019-01-01 14:00:00    | 2999-01-01 00:00:00     | TRUE      |

### Final Select
* Filters for is_active = TRUE and assigns ROW_NUMBER() to product_status_scd_key, ordered by product_status_product_key and scd_start_time.

```sql
-- Select only the active rows for the final SCD result (getting rid of the DELETE rows)
-- Add sequential surrogate key after filtering
SELECT 
    -- ROW_NUMBER() OVER (ORDER BY product_status_product_key, scd_start_time) AS product_status_scd_key, -- SCD Type 2 surrogate key for filtered results
    ROW_NUMBER() OVER (ORDER BY product_status_product_key, cdc_log_position) AS product_status_scd_key,
    product_status_product_key,
    product_status_status,
    scd_start_time,
    scd_end_time
FROM scd_table
WHERE is_active = TRUE
```

| product_status_scd_key | product_status_product_key | product_status_status | scd_start_time         | scd_end_time           |
|-------------------------|----------------------------|-----------------------|-------------------------|-------------------------|
| 1                       | 1                          | 10                    | 2019-01-01 10:00:00    | 2019-01-01 11:00:00     |
| 2                       | 1                          | 20                    | 2019-01-01 11:00:00    | 2019-01-01 12:00:00     |
| 3                       | 1                          | 10                    | 2019-01-01 14:00:00    | 2999-01-01 00:00:00     |


### MERGE operation
  * **MERGE INTO ... AS target USING (...) AS source ON ...**: 
    * A MERGE statement compares the target (existing records in `product_status_hst`) with the source (new/updated records in scd_table). 
    * The matching process uses the following keys: 
      * `product_status_product_key`, `product_status_status`, `scd_start_time`, `scd_end_time`
    * It does not use `product_status_product_key` because if it did, it would wrongly update it in the target depending on the cdc_log_position being used.
      * For example, if we use the condition `cdc_log_position > 1`, than the source would contain only 2 rows and the `product_status_product_key` would go from 1 to 2 (instead of 1 to 3) and it would mess with the target enumeration process.
  * **Update Condition (WHEN MATCHED AND ...cdc_log_position > ...)**: If there is a match and the `cdc_log_position` is greater than the a chosen starting point, it still process all data but it updates **only** the rows after this starting point for the following columns in the target: `product_status_status`, `scd_start_time`, and `scd_end_time`.
  * **Insert Condition**: If there is no match, it inserts the new record with all relevant columns, preserving the SCD history.

### Calling the Stored Procedure

* Process all CDC data: 
    * *CALL CAIOCVELASCO.DATA_ENGINEER.PROCESS_CDC_SCD(0);*

* Process all CDC data, but perform history changes **after** a specific cdc_log_position:
    * *CALL CAIOCVELASCO.DATA_ENGINEER.PROCESS_CDC_SCD(2);*

## Queries

```sql
-- Distinc Products for a given state and date
SELECT DISTINCT product_status_product_key AS product_key
FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_HST
WHERE product_status_status = '10'  -- Replace 'your_state_value' with the actual state you want to test
  AND scd_start_time <= CAST('2019-01-01' AS TIMESTAMP_NTZ) + INTERVAL '1 DAY' - INTERVAL '1 SECOND'  -- Replace '2024-10-29' with your desired test date
  AND scd_end_time >= CAST('2019-01-01' AS TIMESTAMP_NTZ);  -- Replace '2024-10-29' with the same date as above
```

* This query is designed to select distinct products that were in a given state (product_status_status = '10') during a specific date range, with particular focus on historical records of product statuses from the PRODUCT_STATUS_HST table.

Result set:
| product_key |
|-------------|
| 1           |

## Data Quality

### Query 1 - Check for Existence of Product Keys

```sql
-- 1. Check for Existence of Product Keys
-- Ensure that all unique product keys in the CDC table exist in the SCD table.
-- Outcome: If any product keys are returned, it indicates that those keys are missing in the SCD table.
SELECT DISTINCT cdc.product_key
FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_CDC AS cdc
LEFT JOIN CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_HST AS scd
ON cdc.product_key = scd.product_status_product_key
WHERE scd.product_status_product_key IS NULL;
```

Result set:
| product_key |
|-------------|
| EMPTY       |

### Query 2 - Check for Duplicates in SCD

```sql
-- 2. Check for Duplicates in SCD
-- Ensure that there are no duplicate records for the same product key in the SCD table.
-- Outcome: If any rows are returned, it indicates that there are duplicates for those product keys in the SCD table.
SELECT product_status_scd_key, product_status_product_key, product_status_status, scd_start_time, scd_end_time, COUNT(*) AS duplicate_count
FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_HST
GROUP BY product_status_scd_key, product_status_product_key, product_status_status, scd_start_time, scd_end_time
HAVING COUNT(*) > 1;
```

Result set:
| product_status_product_key | product_status_status | scd_start_time | scd_end_time | duplicate_count |
|----------------------------|-----------------------|----------------|--------------|-----------------|
| EMPTY                      | EMPTY                 | EMPTY          | EMPTY        | EMPTY           |

### Query 3 - Cross Check for Active Records with CDC Table


  ```sql
-- 3. Check for Active Records
-- Verify that all active records in the SCD table (where scd_end_time is a future date, e.g., 2999-01-01) 
-- correspond to entries in the CDC table. This ensures that the current status of a product is accurately represented.
-- Outcome: If there are any active records in the SCD table that do not have a matching product key in the CDC table, they will be returned, indicating a potential inconsistency in the data.
SELECT DISTINCT s.product_status_product_key
FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_HST s
LEFT JOIN CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_CDC c
ON s.product_status_product_key = c.product_key
WHERE s.scd_end_time = '2999-01-01 00:00:00' -- Active records
  AND c.product_key IS NULL; -- Check for missing entries in CDC (This condition checks for records in the SCD table that do not have a corresponding entry in the CDC table. If the product_key from the CDC table is NULL, it means that there was no match for that product_status_product_key, indicating that there is an active record in the SCD table that isn’t represented in the CDC table.)
```

Result set:
| product_status_product_key |
|----------------------------|
| EMPTY                      |

### Query 4 - Last Status Consistency Check
* The WHERE clause ensures that we only select the rows where there is a discrepancy between the latest status in the CDC table and the corresponding active status in the SCD table.
* Specifically, the query looks for:
    * No matching status in the SCD table (s.rn <> 1 or s.product_status_status IS NULL).
    * A mismatch between the statuses in the SCD and CDC tables (s.product_status_status <> c.latest_status).
```sql
WITH latest_cdc AS (
    -- Retrieve the latest status for each product key from the CDC table
    SELECT 
        product_key,
        status AS latest_status,
        -- ROW_NUMBER() OVER (PARTITION BY product_key ORDER BY change_time DESC) AS rn
        ROW_NUMBER() OVER (PARTITION BY product_key ORDER BY change_time DESC, cdc_log_position DESC) AS rn -- rn = 1 to the latest status
    FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_CDC
),

latest_scd AS (
    -- Get the latest active status for each product key from the SCD table
    SELECT 
        product_status_product_key,
        product_status_status,
        ROW_NUMBER() OVER (PARTITION BY product_status_product_key ORDER BY scd_start_time DESC) AS rn
        -- ROW_NUMBER() OVER (PARTITION BY product_key ORDER BY change_time DESC, cdc_log_position DESC) AS rn
    FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_HST
    WHERE scd_end_time = '2999-01-01 00:00:00' -- Only consider active records
)

SELECT 
    c.product_key,
    c.latest_status AS cdc_latest_status,
    s.product_status_status AS scd_latest_status
FROM latest_cdc c
LEFT JOIN latest_scd s ON c.product_key = s.product_status_product_key
WHERE c.rn = 1 -- Get the latest status from CDC and combine with the one from SCD
    AND (s.rn <> 1 OR s.product_status_status IS NULL -- Ensure no matching status in SCD or no active status
         OR s.product_status_status <> c.latest_status); -- Check if the status is different
```

Result set:
| product_key |cdc_latest_status |scd_latest_status |
|-------------|------------------|------------------|
| EMPTY       | EMPTY            | EMPTY            |

### Query 5 - Time Range Validation
* This query performs a time range validation check on the SCD table to detect any overlapping records for each product_key. 
* Overlapping records would indicate an issue with the SCD’s time tracking for status changes, as it implies that two records for the same product key are valid at the same time.

```sql
-- 5) Time Range Validation
-- This check calculates the end time of one record and compares it to the start time of the next record for each product key. If the end time exceeds the next start time, it indicates overlapping records.
-- Outcome: No output means that there is no overlapping records.
WITH time_ranges AS (
    SELECT 
        product_status_product_key,
        scd_start_time,
        scd_end_time,
        LEAD(scd_start_time) OVER (PARTITION BY product_status_product_key ORDER BY scd_start_time) AS next_start_time
    FROM CAIOCVELASCO.DATA_ENGINEER.PRODUCT_STATUS_HST
)

SELECT 
    product_status_product_key,
    COUNT(*) AS overlapping_records
FROM time_ranges
WHERE scd_end_time > next_start_time
GROUP BY product_status_product_key
HAVING COUNT(*) > 0;
```

**CTE: time_ranges**
| product_status_product_key | scd_start_time         | scd_end_time           | next_start_time        |
|----------------------------|------------------------|-------------------------|------------------------|
| 1                          | 2019-01-01 10:00:00    | 2019-01-01 11:00:00     | 2019-01-01 11:00:00    |
| 1                          | 2019-01-01 11:00:00    | 2019-01-01 12:00:00     | 2019-01-01 14:00:00    |
| 1                          | 2019-01-01 14:00:00    | 2999-01-01 00:00:00     | NULL                   |

**Final Result:**
| product_status_product_key | overlapping_records |
|----------------------------|---------------------|
| EMPTY                      | EMPTY               |
